In [3]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import scipy.stats as ss, scipy.integrate as integrate

In [5]:
df = pd.read_csv('data/policecalls2022.csv')
df

,CDTS,EID,START_DATE,CALL_NUMBER,PRIORITY,REPORT_DATE,OFFENSE_DATE,OFFENSE_TIME,CALLTYPE_CODE,CALL_TYPE,FINAL_DISPO_CODE,FINAL_DISPO,COMMON_PLACE_NAME,ADDRESS,CITY,STATE
0,20220101042916PS,8913184,1/1/2022 12:00:00 AM,P220010180,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,02:49:02,1066,SUSPICIOUS PERSON,N,No report required; dispatch record only,NaN,[800]-[900] JEANNE AV,San Jose,CA
1,20220101043036PS,8913196,1/1/2022 12:00:00 AM,P220010191,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,03:05:51,415,DISTURBANCE,N,No report required; dispatch record only,NaN,[1500]-[1600] SOUTHWEST EX,San Jose,CA
2,20220101043827PS,8913252,1/1/2022 12:00:00 AM,P220010236,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,04:35:25,911UNK,UNK TYPE 911 CALL,CAN,Canceled,WESTMONT HS,[4800]-[4900] WESTMONT AV,San Jose,CA
3,20220101044026PS,8913216,1/1/2022 12:00:00 AM,P220010206,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,03:38:14,SUSCIR,SUSPICIOUS CIRCUMSTANCES,N,No report required; dispatch record only,NaN,[3600]-[3700] GREENLEE DR,San Jose,CA
4,20220102052501PS,8914182,1/1/2022 12:00:00 AM,P220010965,4,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,23:25:49,11300,NARCOTICS,A,Arrest Made,NaN,E CAPITOL EX & EVERGREEN PL,San Jose,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33037,20220108163647PS,8922559,1/9/2022 12:00:00 AM,P220080519,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,16:27:04,415A,"DISTURBANCE, FIGHT",N,No report required; dispatch record only,EASTRIDGE MALL,[2200]-[2200] EASTRIDGE LP,San Jose,CA
33038,20220108163747PS,8922442,1/9/2022 12:00:00 AM,P220080431,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,14:25:17,242,BATTERY,O,Supplemental report taken,NaN,[100]-[200] BRANHAM LN,San Jose,CA
33039,20220108164003PS,8922553,1/9/2022 12:00:00 AM,P220080513,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,16:22:08,415,DISTURBANCE,N,No report required; dispatch record only,NaN,[3300]-[3400] MT LOGAN DR,San Jose,CA
33040,20220108202855PS,8922752,1/9/2022 12:00:00 AM,P220080662,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,19:46:39,415,DISTURBANCE,G,Gone on Arrival/unable to locate,NaN,[1400]-[1500] KINGMAN AV,San Jose,CA


In [12]:
calltype_codes_with_text = df[['CALLTYPE_CODE', 'CALL_TYPE']].drop_duplicates()
calltype_codes_with_text.to_csv('calltype_codes.csv')

In [ ]:
calltype_codes_with_text

In [10]:
dispo_codes_with_text = df[['FINAL_DISPO_CODE', 'FINAL_DISPO']].drop_duplicates()
dispo_codes_with_text = dispo_codes_with_text.sort_values(by=['FINAL_DISPO_CODE'], axis=0)
dispo_codes_with_text.to_csv('dispo_codes.csv')